<a href="https://colab.research.google.com/github/nguyenvuong1309/Do-an-lap-trinh-an-toan/blob/main/BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import pandas as pd
from numpy import zeros, newaxis
import numpy as np
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional,Activation
import csv


In [3]:
csv_file_path = '/content/drive/MyDrive/Colab/dataset/text_dataset.csv'
df = pd.read_csv(csv_file_path,sep=",")

In [4]:
dataset = df

In [5]:
print(dataset.head())

   Index                                                  1  label
0      0  1 150200/dirent_uri.c memset 405\nconst char *...      0
1      1  2 79848/CWE134_Uncontrolled_Format_String__cha...      0
2      2  3 94203/CWE319_Cleartext_Tx_Sensitive_Info__w3...      0
3      3  4 151133/utils.c strcmp 737\nint avformat_open...      0
4      4  5 94266/CWE319_Cleartext_Tx_Sensitive_Info__w3...     32


In [6]:
dataset = dataset[['1','label']]
dataset.head()

,1,label
0,1 150200/dirent_uri.c memset 405\nconst char *...,0
1,2 79848/CWE134_Uncontrolled_Format_String__cha...,0
2,3 94203/CWE319_Cleartext_Tx_Sensitive_Info__w3...,0
3,4 151133/utils.c strcmp 737\nint avformat_open...,0
4,5 94266/CWE319_Cleartext_Tx_Sensitive_Info__w3...,32


In [7]:
print(dataset['label'].unique())
print(len(dataset['label'].unique()))

[ 0 32  4 10 20 36  3  1 12 25 15 34 16 11  2 14 35 38  9 29  7 18 37 21
 22 23  5  8 17 31 39 33 28 30 19 40 26 13 24 27  6]
41


In [8]:
train_dataset = dataset[0:int(len(dataset) * 0.8)]
test_dataset =  dataset[int(len(dataset) * 0.8) :]

In [9]:
#<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
text_data = train_dataset['1'].values
tensor = tf.data.Dataset.from_tensor_slices(text_data)

In [10]:
tensor

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [11]:
train_dataset_numpy = np.array(train_dataset)

In [12]:
print(train_dataset_numpy[:3])

[['1 150200/dirent_uri.c memset 405\nconst char *svn_relpath__internal_style(const char *relpath,apr_pool_t *pool) 861\nreturn svn_relpath_canonicalize ( internal_style ( relpath , pool ) , pool ) ; 863\nconst char *svn_relpath_canonicalize(const char *relpath,apr_pool_t *pool) 1411\nreturn canonicalize ( type_relpath , relpath , pool ) ; 1413\nstatic const char *canonicalize(path_type_t type,const char *path,apr_pool_t *pool) 389\nchar * canon ; 391\nchar * dst ; 392\nif ( path [ 0 ] == \'\\0\' )  401\ndst = canon = ( memset ( apr_palloc ( pool , strlen ( path ) + 1 ) , 0 , strlen ( path ) + 1 ) ); 405\nwhile ( ( * src ) != \':\' )  419\n* ( dst ++ ) = canonicalize_to_lower ( * ( src ++ ) ); 420\n* ( dst ++ ) = \':\'; 423\n* ( dst ++ ) = 47; 424\n* ( dst ++ ) = 47; 425\nsrc += 3; 426\nseg = src; 429\nwhile ( * src && ( * src ) != 47 && ( * src ) != 64 )  430\nsrc ++; 431\nif ( ( * src ) == 64 )  432\nseglen = ( src - seg + 1 ); 434\nmemcpy ( dst , seg , seglen ); 435\ndst += seglen; 4

In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [14]:
train_dataset = train_dataset.sample(frac=0.5, replace=True, random_state=1)
test_dataset = test_dataset.sample(frac=0.5, replace=True, random_state=1)

In [15]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(tensor)

In [16]:
train_dataset['1']

128037    128038 65080/CWE121_Stack_Based_Buffer_Overflo...
5192      5193 80996/CWE134_Uncontrolled_Format_String__...
50057     50058 63206/CWE121_Stack_Based_Buffer_Overflow...
109259    109260 152935/config_file.c strlen 925\ninline...
73349     73350 117223/CWE789_Uncontrolled_Mem_Alloc__ma...
                                ...                        
85221     85222 93409/CWE256_Plaintext_Storage_of_Passwo...
17921     17922 99720/CWE401_Memory_Leak__malloc_realloc...
113980    113981 149951/color.c strlen 402\nvoid parse_d...
117534    117535 117207/CWE789_Uncontrolled_Mem_Alloc__m...
40580     40581 72811/CWE122_Heap_Based_Buffer_Overflow_...
Name: 1, Length: 72000, dtype: object

In [17]:
train_dataset

,1,label
128037,128038 65080/CWE121_Stack_Based_Buffer_Overflo...,0
5192,5193 80996/CWE134_Uncontrolled_Format_String__...,0
50057,50058 63206/CWE121_Stack_Based_Buffer_Overflow...,0
109259,109260 152935/config_file.c strlen 925\ninline...,22
73349,73350 117223/CWE789_Uncontrolled_Mem_Alloc__ma...,0
...,...,...
85221,85222 93409/CWE256_Plaintext_Storage_of_Passwo...,0
17921,17922 99720/CWE401_Memory_Leak__malloc_realloc...,0
113980,113981 149951/color.c strlen 402\nvoid parse_d...,0
117534,117535 117207/CWE789_Uncontrolled_Mem_Alloc__m...,0


In [18]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', '0', 'if', 'data', '1', 'char', 'void', 'i', 'd', 's',
       'service', 'sizeof', 'int', '100', 'recvresult', '2', 'static',
       'datalen', 'null'], dtype='<U559')

In [19]:
example = [[' /CWE_Heap_Based_Buffer_Overflow__cpp_CWE_char_snprintf_.cpp memset \nvoid bad() \nchar * data ; \ndata = new char [  ]; \nif ( GLOBAL_CONST_TRUE )  \nmemset ( data , \'A\' ,  -  ); \ndata [  -  ] = \'\\\'; \nSNPRINTF ( dest , strlen ( data ) , "%s" , data ); \nprintLine ( data ); \nvoid printLine (const char * line) \nif ( line != NULL )  \nprintf ( "%s\\n" , line ); \ndelete [ ] data \n\n'],
 [' /CWE_Trapdoor__ip_based_logic_.c send \nvoid CWE_Trapdoor__ip_based_logic__bad() \nWSADATA wsaData ; \nstruct sockaddr_in service , acceptService ; \nif ( WSAStartup ( MAKEWORD (  ,  ) , & wsaData ) != NO_ERROR )  \nlistenSocket = socket ( AF_INET , SOCK_STREAM , IPPROTO_TCP ); \nif ( listenSocket == INVALID_SOCKET )  \nmemset ( & service ,  , sizeof ( service ) ); \nservice . sin_family = AF_INET; \nservice . sin_addr . s_addr = INADDR_ANY; \nservice . sin_port = htons (  ); \nif ( bind ( listenSocket , ( struct sockaddr * ) & service , sizeof ( service ) ) == SOCKET_ERROR )  \nif ( listen ( listenSocket ,  ) == SOCKET_ERROR )  \nacceptSocket = accept ( listenSocket , NULL , NULL ); \nif ( acceptSocket == SOCKET_ERROR )  \nif ( getsockname ( acceptSocket , ( struct sockaddr * ) & acceptService , & acceptServiceLen ) == -  )  \nif ( strcmp ( "..." , inet_ntoa ( acceptService . sin_addr ) ) ==  )  \nif ( send ( acceptSocket , ADMIN_MESSAGE , strlen ( ADMIN_MESSAGE ) ,  ) == SOCKET_ERROR )  \nif ( send ( acceptSocket , DEFAULT_MESSAGE , strlen ( DEFAULT_MESSAGE ) ,  ) == SOCKET_ERROR )  \nwhile (  )  \n\n'],
 [' /CWE_OS_Command_Injection__char_console_wspawnl_.c strlen \nvoid CWE_OS_Command_Injection__char_console_wspawnl__bad() \nchar * data ; \nchar dataBuffer [  ] = "" ; \ndata = dataBuffer; \nif ( staticFive ==  )  \nsize_t dataLen = strlen ( data ) ; \nif (  - dataLen >  )  \nif ( fgets ( data + dataLen , ( int ) (  - dataLen ) , stdin ) != NULL )  \ndataLen = strlen ( data ); \nif ( dataLen >  && data [ dataLen -  ] == \'\\n\' )  \ndata [ dataLen -  ] = \'\\\'; \n\n']
           ]

In [20]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[   1,   34,    7,  170,    6,    4,    4,  268,    6,    3,  820,
          34,    4,  111,    4,  301,   43,   29,    4,   10,    4,   54,
           4,    7,   54,   35,    6,   20,    3,   20,   19,   69,  104,
          20,  385,    4,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   1, 1532,    7,    1,   28,   28,   24,   85,   11, 5859,    3,
          88,   89,   28,   94,   32,   68,   45,   86,   95,    3,   32,
          63,   34,   11,   12,   11,   11,   91,   45,   11,   84,   83,
         184,   11,   87,   90,    3,  178,   32,   24,   80,   11,   12,
          11,   26,    3,  182,   32,   26,   47,  181,   32,   19,   19,
           3,   47,   26,    3, 7586,   47,   24,   80,

In [21]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(41, activation='softmax')
])

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [23]:
X_train = train_dataset['1']
y_train = train_dataset['label']
y_train = tf.keras.utils.to_categorical(y_train, num_classes=41)

In [24]:
print(X_train.shape)
print(y_train.shape)

(72000,)
(72000, 41)


In [25]:
history = model.fit(X_train,y_train, epochs=1,
                    # validation_data=test_dataset,
                    # validation_steps=30
                    )

2250/2250 [==============================] - 3500s 2s/step - loss: 1.0652 - accuracy: 0.7952


In [26]:
X_test = test_dataset['1']
y_test = test_dataset['label']
y_test = tf.keras.utils.to_categorical(y_test, num_classes=41)

In [27]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

563/563 [==============================] - 168s 285ms/step - loss: 0.6521 - accuracy: 0.8667
Test Loss: 0.6521329879760742
Test Accuracy: 0.8667222261428833
